# MongoDB Atlas | MongoDB Atlas

[MongoDB Atlas](https://www.mongodb.com/)は、ベクターデータベースとして使用できるドキュメント型データベースです。

> [MongoDB Atlas](https://www.mongodb.com/) is a document database that can be
> used as a vector databse.

このウォークスルーでは、`MongoDB Atlas` ベクターストアを使用した `SelfQueryRetriever` のデモを行います。

> In the walkthrough, we'll demo the `SelfQueryRetriever` with a `MongoDB Atlas` vector store.



## Creating a MongoDB Atlas vectorstore | MongoDB Atlas ベクターストアの作成

まず、MongoDB Atlas VectorStoreを作成し、いくつかのデータを投入して初期化します。映画の要約が含まれた小さなデモ用ドキュメントセットを作成しました。

> First we'll want to create a MongoDB Atlas VectorStore and seed it with some data. We've created a small demo set of documents that contain summaries of movies.

注意：self-query retrieverを使用するには、`lark`がインストールされている必要があります（`pip install lark`でインストールできます）。また、`pymongo`パッケージも必要です。

> NOTE: The self-query retriever requires you to have `lark` installed (`pip install lark`). We also need the `pymongo` package.



In [5]:
#!pip install lark pymongo

`OpenAIEmbeddings`を使用するためには、OpenAI APIキーを取得する必要があります。

> We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.



In [ ]:
import os

OPENAI_API_KEY = "Use your OpenAI key"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import MongoDBAtlasVectorSearch
from pymongo import MongoClient

CONNECTION_STRING = "Use your MongoDB Atlas connection string"
DB_NAME = "Name of your MongoDB Atlas database"
COLLECTION_NAME = "Name of your collection in the database"
INDEX_NAME = "Name of a search index defined on the collection"

MongoClient = MongoClient(CONNECTION_STRING)
collection = MongoClient[DB_NAME][COLLECTION_NAME]

embeddings = OpenAIEmbeddings()

In [ ]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "action"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "genre": "thriller", "rating": 8.2},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "rating": 8.3, "genre": "drama"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={"year": 1979, "rating": 9.9, "genre": "science fiction"},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "genre": "thriller", "rating": 9.0},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated", "rating": 9.3},
    ),
]

vectorstore = MongoDBAtlasVectorSearch.from_documents(
    docs,
    embeddings,
    collection=collection,
    index_name=INDEX_NAME,
)

それでは、クラスターにベクター検索インデックスを作成しましょう。以下の例では、`embedding`は埋め込みベクターを含むフィールドの名前です。Atlas Vector Search インデックスの定義方法の詳細については、[ドキュメント](https://www.mongodb.com/docs/atlas/atlas-search/field-types/knn-vector)を参照してください。
インデックスには`{COLLECTION_NAME}`という名前を付け、`{DB_NAME}.{COLLECTION_NAME}`の名前空間でインデックスを作成します。最後に、MongoDB AtlasのJSONエディタに以下の定義を書き込んでください：

> Now, let's create a vector search index on your cluster. In the below example, `embedding` is the name of the field that contains the embedding vector. Please refer to the [documentation](https://www.mongodb.com/docs/atlas/atlas-search/field-types/knn-vector) to get more details on how to define an Atlas Vector Search index.
> You can name the index `{COLLECTION_NAME}` and create the index on the namespace `{DB_NAME}.{COLLECTION_NAME}`. Finally, write the following definition in the JSON editor on MongoDB Atlas:

```json
{
  "mappings": {
    "dynamic": true,
    "fields": {
      "embedding": {
        "dimensions": 1536,
        "similarity": "cosine",
        "type": "knnVector"
      },
      "genre": {
        "type": "token"
      },
      "ratings": {
        "type": "number"
      },
      "year": {
        "type": "number"
      }
    }
  }
}
```



## Creating our self-querying retriever | 自己問い合わせリトリバーの作成

これで、私たちのリトリーバーをインスタンス化することができます。これを行うには、ドキュメントがサポートしているメタデータフィールドに関する情報と、ドキュメントの内容の簡潔な説明を事前に提供する必要があります。

> Now we can instantiate our retriever. To do this we'll need to provide some information upfront about the metadata fields that our documents support and a short description of the document contents.



In [ ]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"

In [ ]:
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

## Testing it out | テストしてみる

それでは、実際に私たちのリトリーバーを使ってみましょう！

> And now we can try actually using our retriever!



In [ ]:
# This example only specifies a relevant query
retriever.get_relevant_documents("What are some movies about dinosaurs")

In [ ]:
# This example specifies a filter
retriever.get_relevant_documents("What are some highly rated movies (above 9)?")

In [ ]:
# This example only specifies a query and a filter
retriever.get_relevant_documents(
    "I want to watch a movie about toys rated higher than 9"
)

In [ ]:
# This example specifies a composite filter
retriever.get_relevant_documents(
    "What's a highly rated (above or equal 9) thriller film?"
)

In [ ]:
# This example specifies a query and composite filter
retriever.get_relevant_documents(
    "What's a movie after 1990 but before 2005 that's all about dinosaurs, \
    and preferably has a lot of action"
)

## Filter k | フィルター k

さらに、self query retrieverを使用して、取得するドキュメントの数`k`を指定することもできます。

> We can also use the self query retriever to specify `k`: the number of documents to fetch.

これを行うには、コンストラクタに `enable_limit=True` を渡すことで実現できます。

> We can do this by passing `enable_limit=True` to the constructor.



In [ ]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=True,
    enable_limit=True,
)

In [ ]:
# This example only specifies a relevant query
retriever.get_relevant_documents("What are two movies about dinosaurs?")